In [20]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [21]:
import pandas as pd

#unfiltered_subjects = ['Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
unfiltered_subjects = ['Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
ch_names = ['timestamps','Fp1','Fp2','C3','C4','P7','P8','O1','O2','F7','F8','F3','F4','T7','T8','P3','P4','stim','sfreq']
#unfiltered_subjects = ['Aanand_19_02.csv']
# Create an empty DataFrame to store all data
all_data = pd.DataFrame(columns=ch_names)

for file_name in unfiltered_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 0) & (df['timestamps'] < 150)].copy()
    lvl2_sub = df[(df['timestamps'] >= 150) & (df['timestamps'] < 280)].copy()
    lvl3_sub = df[(df['timestamps'] >= 280) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    all_data = pd.concat([all_data, combined_df])

C:\Users\LEGION\AppData\Local\Temp\ipykernel_17896\2273160114.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, combined_df])


In [22]:
drop1 = all_data.drop(['sfreq'], axis=1)
df = drop1.drop(['stim'], axis=1)


#drop_data = testing_data.drop(['sfreq'], axis=1)
# df2 = drop_data.drop(['stim'], axis=1)

In [23]:
df.shape

(636251, 18)

In [24]:
label_counts = df['label'].value_counts()
print(label_counts)

df1 = df

label
1.0    243767
2.0    211038
3.0    181446
Name: count, dtype: int64


In [25]:
df = df1.drop(['label'], axis=1)

In [26]:
eeg_data = df.iloc[:,1:18].values.T
eeg_data[:-1] = eeg_data[:-1] * (10*(-6))
eeg_data[:-1] = eeg_data[:-1] / 4

ch_names = ['Fp1',	'Fp2'	,'C3'	,'C4'	,'P7'	,'P8'	,'O1'	,'O2'	,'F7'	,'F8'	,'F3'	,'F4'	,'T7'	,'T8'	,'P3'	,'P4']
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = ( ["eeg" ]*16))
raw = mne.io.RawArray(eeg_data, info)
raw.set_eeg_reference()

raw.filter(l_freq = 1,h_freq = 30)

Creating RawArray with float64 data, n_channels=16, n_times=636251
    Range : 0 ... 636250 =      0.000 ...  5302.083 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.6s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [27]:
duration = 0.6
epochs=mne.make_fixed_length_epochs(raw, duration = duration, overlap=0,preload=True)

Not setting metadata
8836 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 8836 events and 72 original time points ...
0 bad epochs dropped


In [28]:
epochs.drop_bad()  # This will drop bad epochs
num_epochs = len(epochs)
print("Total number of epochs created:", num_epochs)

Total number of epochs created: 8836


In [29]:
epochs.get_data()

C:\Users\LEGION\AppData\Local\Temp\ipykernel_17896\1251544707.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs.get_data()


array([[[ 1.81898940e-11, -9.47492918e+04, -1.18740849e+05, ...,
         -1.17464478e+04, -1.05092815e+04, -9.50704713e+03],
        [ 3.43334250e-11, -8.71324937e+04, -1.09063184e+05, ...,
         -1.07292794e+04, -9.49589701e+03, -8.59697282e+03],
        [-1.20508048e-11, -4.18075628e+03, -3.32758466e+03, ...,
         -5.07563124e+02, -3.45659406e+00,  2.69526119e+02],
        ...,
        [ 7.27595761e-12,  1.05012965e+06,  1.34087222e+06, ...,
          1.30319657e+05,  1.21916098e+05,  1.13793069e+05],
        [ 2.27373675e-12,  2.93703433e+04,  3.14944841e+04, ...,
          3.18296881e+03,  2.20649428e+03,  1.62521538e+03],
        [ 8.73114914e-11, -5.94392622e+05, -7.53669296e+05, ...,
         -7.30744418e+04, -6.87344923e+04, -6.44681279e+04]],

       [[-9.10366815e+03, -8.89192033e+03, -8.20868110e+03, ...,
          3.47772800e+03,  3.07556585e+03,  2.52048703e+03],
        [-8.34702527e+03, -8.21720739e+03, -7.52975089e+03, ...,
          3.36424393e+03,  2.93096643e

In [30]:
import mne

eeg_data = epochs.get_data()

condition_ranges = {'lev1': (0, (num_epochs/3)),
                    'lev2': ((num_epochs/3) , 2*((num_epochs)/3) ),
                    'lev3': ( 2*((num_epochs)/3) , (num_epochs) )}

epochs_by_condition = {}
selected_epochs = []

for condition, (start_epoch, end_epoch) in condition_ranges.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = epochs[start_epoch:end_epoch+1].copy()
    epochs_by_condition[condition] = condition_epochs
    selected_epochs.append(condition_epochs)

selected_epochs = mne.concatenate_epochs(selected_epochs)

for condition, epoch_data in epochs_by_condition.items():
    num_epochs = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs}")

print(f"Total number of selected epochs: {len(selected_epochs)}")


C:\Users\LEGION\AppData\Local\Temp\ipykernel_17896\3628746636.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  eeg_data = epochs.get_data()


Not setting metadata
8838 matching events found
No baseline correction applied
Number of epochs for lev1: 2946
Number of epochs for lev2: 2946
Number of epochs for lev3: 2946
Total number of selected epochs: 8838


In [31]:

condition_of_interest = 'lev1'
epoch_array_for_condition = epochs_by_condition[condition_of_interest]
print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


Epoch array shape for condition lev1: (2946, 16, 72)


C:\Users\LEGION\AppData\Local\Temp\ipykernel_17896\1489624503.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


In [32]:
import pandas as pd
import numpy as np
from scipy.signal import welch



def calculate_psd_features(epoch_data, sfreq):
    psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
    psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
    psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
    psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

    freq_min = 14
    freq_max = 30

    # Find the indices corresponding to the frequency range of interest
    idx_min = np.argmax(psd_frequencies > freq_min) - 1
    idx_max = np.argmax(psd_frequencies > freq_max)

    # Extract the PSD within the specified frequency range
    psd_values_alpha_band = psd_values[:, idx_min:idx_max]
    psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
    psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
    psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
    return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band

channel_names = epochs.ch_names
sfreq = epochs.info['sfreq']  # Sampling frequency

columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
columns.append('condition')
feature_df = pd.DataFrame(columns=columns)

def append_epoch_data(epoch_array, condition_label):
    global feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=columns)
        feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

append_epoch_data(epochs_by_condition['lev1'], 'lev1')
append_epoch_data(epochs_by_condition['lev2'], 'lev2')
append_epoch_data(epochs_by_condition['lev3'], 'lev3')

print("Shape of feature DataFrame:", feature_df.shape)
print(feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 72, using nperseg = 72
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of feature DataFrame: (8838, 49)
         mean_psd_Fp1       mean_psd_Fp2         mean_psd_C3  \
0   5512.369895133467  7468.764680506607  4578.8885067972615   
1   4234.727236419005  5870.520673380197   4043.936215107492   
2  3800.1082573177932  5408.885491299386   3377.761748504354   
3  4606.4898499409765  5050.572755186192   3450.141006539079   
4  2600.6593104103436  4272.618055791072   2993.663082898885   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0  2835.5387849238946   6350.195010816803  4859.4610882782845   
1  2452.9395267930727   4927.823978237707   4308.861778029582   
2  2024.3218713156114  3658.0347706777616  3427.3149541238968   
3  1899.5756042208568  3784.4448887433027   3626.653281937911   
4  1773.1110979720042   3074.766775518717   2625.900341410021   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   7079.589898375131   4271.797098756459   3549.619212728695   
1   9540.018561123077  3576.1924305012435  3105.6258140

In [33]:
# Rename columns using a dictionary
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

# Rename columns using the rename() method
feature_df.rename(columns=new_column_names, inplace=True)

# Print the DataFrame
print(feature_df.head())


         mean_psd_Fp1       mean_psd_Fp2         mean_psd_C3  \
0   5512.369895133467  7468.764680506607  4578.8885067972615   
1   4234.727236419005  5870.520673380197   4043.936215107492   
2  3800.1082573177932  5408.885491299386   3377.761748504354   
3  4606.4898499409765  5050.572755186192   3450.141006539079   
4  2600.6593104103436  4272.618055791072   2993.663082898885   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0  2835.5387849238946   6350.195010816803  4859.4610882782845   
1  2452.9395267930727   4927.823978237707   4308.861778029582   
2  2024.3218713156114  3658.0347706777616  3427.3149541238968   
3  1899.5756042208568  3784.4448887433027   3626.653281937911   
4  1773.1110979720042   3074.766775518717   2625.900341410021   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   7079.589898375131   4271.797098756459   3549.619212728695   
1   9540.018561123077  3576.1924305012435  3105.6258140918358   
2  4033.9871105526818  2893.0

In [34]:
feature_df

,mean_psd_Fp1,mean_psd_Fp2,mean_psd_C3,mean_psd_C4,mean_psd_P7,mean_psd_P8,mean_psd_O1,mean_psd_O2,mean_psd_F7,mean_psd_F8,...,stddev_psd_O2,stddev_psd_F7,stddev_psd_F8,standarddev_psd_F3,standarddev_psd_F4,standarddev_psd_T7,standarddev_psd_T8,standarddev_psd_P3,standarddev_psd_P4,condition
0,5512.369895133467,7468.764680506607,4578.8885067972615,2835.5387849238946,6350.195010816803,4859.4610882782845,7079.589898375131,4271.797098756459,3549.619212728695,9176.999350314976,...,8282.9169611084,5929.9629943869395,18481.597126613553,21858.117437977256,3777.0897268642657,21503.732773410316,31996.53661819277,22696.283793872986,16659.3064521424,lev1
1,4234.727236419005,5870.520673380197,4043.936215107492,2452.9395267930727,4927.823978237707,4308.861778029582,9540.018561123077,3576.1924305012435,3105.6258140918358,8067.570836622922,...,6996.919626438105,5401.825081102116,15260.659426082988,18494.286307198436,3201.9634209852907,18549.151013310042,2487.371844500067,19330.958544694062,2162.140285683822,lev1
2,3800.1082573177932,5408.885491299386,3377.761748504354,2024.3218713156114,3658.0347706777616,3427.3149541238968,4033.9871105526818,2893.0944790323165,2375.9406927068376,6814.440864621692,...,5289.308875257017,3687.0598571262185,13214.47954625298,16529.378128367418,2029.3964641283278,16151.99542238811,1472.584601404371,16820.609850698263,1317.1401770684179,lev1
3,4606.4898499409765,5050.572755186192,3450.141006539079,1899.5756042208568,3784.4448887433027,3626.653281937911,4956.599330244578,2950.401929694715,2697.575284302093,5768.828273322669,...,5839.634463252912,5007.553142159767,11306.1271380822,13768.108128719525,3881.330228709602,13578.681772936594,2347.975811120148,13790.030608309058,2635.244382866,lev1
4,2600.6593104103436,4272.618055791072,2993.663082898885,1773.1110979720042,3074.766775518717,2625.900341410021,5415.080899546827,2155.117823487809,1922.7232970920675,5743.52068117045,...,4170.000211188462,2694.4199910351153,11138.277029997074,13542.806052126021,1351.724743594396,14016.754270053347,1227.597757497885,14545.22012723503,960.746833299487,lev1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,1217.392144797381,536.6006466132942,533.7706704661942,506.9305714946208,468.63786024052155,527.1365018935895,538.1521891867066,681.3634351802303,739.8767169931524,382.81455320866405,...,683.8082213253462,576.3555335588746,298.10915013556763,267.0167623838261,362.4221644906562,180.33497311851434,188.5578139720356,220.01774065431897,349.5058622432391,lev3
8834,1857.9319753920129,569.2408213853579,507.1033169847787,552.3398609009159,489.6862695215117,624.8928546895777,647.3940568892332,654.7677212655502,781.0861354094119,245.94113342760238,...,903.9780945493501,646.3261331991115,418.33144278302234,306.6561889227734,607.9151956487825,331.18174388683946,291.5886212574139,226.97617175161056,367.68594913194374,lev3
8835,3230.0058279248115,817.9697216015319,1280.8880435417077,1219.8870967618673,886.2222338280361,780.2413237370176,921.2729031056932,1621.0400011165962,1279.1194477736701,502.6031670306042,...,2075.0202694868276,1252.7400342783021,529.1891138460061,318.53772672642725,745.4437665961347,315.95808815302644,551.6101232060892,491.50474327881074,347.1050264608689,lev3
8836,43460.08036908209,8902.120990139198,18714.44542947695,10398.107248950017,7798.599269065371,7235.974626254595,8785.065109112957,11477.81301550319,6035.106513583739,2882.734772068307,...,10150.371930608537,5055.487644421349,1732.6944726185004,1859.6106779417987,267748.97508836584,2460.191117584588,5930.112717912865,8013.867904348029,4114.239276316176,lev3


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9304298642533937


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

gb_classifier.fit(X_train, y_train)

y_pred = gb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9134615384615384


In [37]:
# def append_epoch_data(epoch_array, condition_label):
#         global feature_df
#         for epoch in epoch_array:
#             psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
#             epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
#             epoch_features = np.append(epoch_features, condition_label)
#             epoch_df = pd.DataFrame([epoch_features], columns=columns)
#             feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

# def calculate_psd_features(epoch_data, sfreq):
#         psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
#         psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
#         psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
#         psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

#         freq_min = 14
#         freq_max = 30

#         # Find the indices corresponding to the frequency range of interest
#         idx_min = np.argmax(psd_frequencies > freq_min) - 1
#         idx_max = np.argmax(psd_frequencies > freq_max)

#         # Extract the PSD within the specified frequency range
#         psd_values_alpha_band = psd_values[:, idx_min:idx_max]
#         psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
#         psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
#         psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
#         return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band


In [38]:
# import matplotlib.pyplot as plt

# # Initialize lists to store durations and accuracies
# durations = []
# accuracies = []

# # Define a range of durations to try
# duration_range = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch



    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']  # Sampling frequency

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     # Rename columns using the rename() method
#     feature_df.rename(columns=new_column_names, inplace=True)
#     # Ensure the rest of your code is within this loop so that it executes for each duration
    
#     # Train the classifier and calculate accuracy
#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     rf_classifier.fit(X_train, y_train)
#     y_pred = rf_classifier.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     # Append duration and accuracy to lists
#     durations.append(duration)
#     accuracies.append(accuracy)

# # Plot duration vs accuracy
# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()
